# Finding game pitchers

The purpose of this notebook is to take the `mlb_games_df.csv` file and substitute the pitcher IDs (for some site) instead of name/whatever ID is currently showing up.

In [2]:
import pandas as pd
import numpy as np

import pybaseball as pyb

In [3]:
pyb.playerid_lookup('Tejeda')

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,tejeda,anderson,660731,tejea001,tejedan01,-1,2020.0,2020.0
1,tejeda,rob,434565,tejer001,tejedro01,2575,2005.0,2011.0
